In [10]:
import csv
from bs4 import BeautifulSoup

name = "SDNY FULL_2FLY"
file_type = '.htm'

html_doc = open("data/" + name + file_type)

soup = BeautifulSoup(html_doc, 'html.parser')

tr_list = soup.findAll('tr')
git st
print(len(tr_list))

for thing in tr_list:
    u_list = thing.find_all('u')
    for u in u_list:
        print(u.getText())

841
Defendant                                 (1)
Pending Counts
Disposition
Highest Offense Level (Opening)
Terminated Counts
Disposition
Highest Offense Level (Terminated)
Complaints
Disposition
Defendant                                 (2)
Pending Counts
Disposition
Highest Offense Level (Opening)
Terminated Counts
Disposition
Highest Offense Level (Terminated)
Complaints
Disposition
Defendant                                 (3)
Pending Counts
Disposition
Highest Offense Level (Opening)
Terminated Counts
Disposition
Highest Offense Level (Terminated)
Complaints
Disposition
Defendant                                 (4)
Pending Counts
Disposition
Highest Offense Level (Opening)
Terminated Counts
Disposition
Highest Offense Level (Terminated)
Complaints
Disposition
Defendant                                 (5)
Pending Counts
Disposition
Highest Offense Level (Opening)
Terminated Counts
Disposition
Highest Offense Level (Terminated)
Complaints
Disposition
Defendant                      

In [11]:
# Create a flag to determine which text block
def iterate_through_html():
    
    which_block = None
    pending_counts = []
    terminated_counts = []
    
    pend_disp = []
    term_disp = []
    
    underlined_text = ""
    
    for i in range(len(tr_list)):
        tr = tr_list[i]
        
        u = tr.find('u')
        
        title = False
        if u != None:
            title = True
        
        if title:
            underlined_text = u.getText()

        if title and "Defendant" in underlined_text:
            which_block = None
            
            print(underlined_text)

            # Get information from name block
            if i + 1 >= len(tr_list):
                continue
            name_block = tr_list[i + 1]

            if name_block is None:
                continue
            
            name_element = name_block.find('b')
            if name_element is None:
                continue
            
            name = name_element.getText()
            terminated = name_block.find('i').getText() 
            
            if "TERMINATED" not in terminated:
                terminated = "ONGOING"
            
            print(name)
            
            tds = name_block.find_all('td')
            rep_info = str(tds[len(tds) - 1]).replace('\n','n')
            # End Name Block
        
        elif not title and which_block == "pending":
            # Get info from pending count
            disposition_block = tr_list[i]
            
            if disposition_block is None:
                continue
            
            if("None" in disposition_block.find('td').getText()):
                pending_counts.append(None)
                pend_disp.append(None)
            else:
                # Append pending count, and then the dispostition of count
                pending_counts.append(disposition_block.find_all('td')[0].getText())
                pend_disp.append(disposition_block.find_all('td')[2].getText())
            
        elif not title and which_block == "terminated":
            # Get info from terminated count
            terminated_block = tr_list[i]
            
            if terminated_block is None:
                continue
                
            if("None" in terminated_block.find('td').getText()):
                terminated_counts.append(None)
                term_disp.append(None)
            else:
                # Append terminated count, and then the dispostition of count
                terminated_counts.append(terminated_block.find_all('td')[0].getText())
                term_disp.append(terminated_block.find_all('td')[2].getText())
        
        # Set the mode for the next pass through the tr list, and return the previoius defendant 
        if title and "Pending Counts" in underlined_text:
            which_block = "pending"
        elif title and "Terminated Counts" in underlined_text:
            which_block = "terminated"
        elif title and "Highest" in underlined_text:
            which_block = "highest"
        elif title and "Complaints" in underlined_text:
            # Reached final block in defendant
            which_block = "complaints"
            
            # Tempterarily save the lists of counts
            counts1 = list(pending_counts)
            counts2 = list(terminated_counts)
            
            disp1 = list(pend_disp)
            disp2 = list(term_disp)
                    
            # Reset counts
            pending_counts = []
            terminated_counts = []
            
            pend_disp = []
            term_disp = []
            
            defendant_info = [name, terminated.replace("TERMINATED: ", ""), rep_info]
            
            yield defendant_info, counts1, disp1, counts2, disp2

In [19]:
import csv
import re
            
def list_to_string(listthing):
    string = ""
    for thing in listthing:
        if thing is None:
            thing = ""
        string += thing + ' '
        
    return [string]

csv_file = 'output/' + name + '.csv'

print(csv_file)

with open(csv_file, "w", newline="") as csvfile:
    writer = csv.writer(csvfile, delimiter=",")
    writer.writerow(["Name", "Terminated", "Representative Info", "Pending Counts", "Months", "Pending Disposition", "Terminated Counts", "Terminated Disposition"])
    
    for defendant_info, pending_counts, pend_disp, term_counts, term_disp in iterate_through_html():
#         for i in range(0, max(len(pending_counts), len(term_counts)), 2):
#             # Pad with empty columns if necessary
#             if i < len(pending_counts):
                
# #                 print(pending_counts[i+1])
                
#                 num = None
    
#                 if pending_counts[i+1] is not None:
#                     num = re.search('[0-9]+', pending_counts[i+1])
                
#                 if num is None:
#                     num = '0'
#                 else:
#                     num = num.group()
            
#                 a, b, c = pending_counts[i], num, pending_counts[i + 1]
#             else:
#                 a, b, c = None, None, None
                
#             if i < len(term_counts):
#                 d, e = term_counts[i], term_counts[i + 1]
#             else:
#                 d, e = None, None
            
#             if i > 0:
#                 defendant_info = [None, None, None]

        num = None

        if len(pend_disp) > 0 and pend_disp[0] is not None:
            num = re.search('[0-9]+', pend_disp[0])

        if num is None:
            num = '0'
        else:
            num = num.group()

        row = defendant_info + list_to_string(pending_counts) + [num] + list_to_string(pend_disp) + list_to_string(term_counts) + list_to_string(term_disp)
        writer.writerow(row)

        
        

output/SDNY FULL_2FLY.csv
Defendant                                 (1)
Laquan Parrish
['Laquan Parrish', '10/27/2017', '<td valign="top" width="40%"><b>Nathaniel            Z               Marmur                                            </b>n<br/>The Law Offices of Nathaniel Z. Marmur, PLLC                n<br/>500 5th Avenue                                              n<br/>New York, NY 10110          n<br/>212-257-4894             n<br/>Email: nmarmur@marmurlaw.comn<br/><i>LEAD ATTORNEY</i>n<br/><i>ATTORNEY TO BE NOTICED</i>n<br/><i>Designation: CJA Appointment</i></td>', ' 18:1962-7480.F RACKETEERING (RICO CONSPIRACY)(1)  18:924C.F USE OF FIREARMS IN FURTHERANCE OF COUNTS ONE AND TWO(4) ', '135', 'Imprisonment: 135 Months on Count (S1) One and 60 months on Count (S1) Four, the terms to run consecutively for an aggregate term of imprisonment of 195 months. Supervised Release: 3 Years with the following special condition of supervision: The defendant shall submit his person, resid